In [1]:
import ast
import os
import re
import unicodedata

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from ckiptagger import (
    data_utils, construct_dictionary, 
    WS, POS, NER
)

pd.set_option('display.max_colwidth', -1)

%matplotlib inline

/home/bird/Documents/Code/tbrain20_AML/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bird/Documents/Code/tbrain20_AML/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bird/Documents/Code/tbrain20_AML/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bird/Document

In [2]:
data_path = "data/tbrain_train_final_0610.csv"

df = pd.read_csv(data_path)
df.head()

,news_ID,hyperlink,content,name
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35facebook commentFONT SIZEICON PRINT78981DD6-B6FA-435C-B969-0D6ABD01805C0「時間序列與量化交易研討會」邀集專家學者進行量化交易、智能投資等相關議題研討。(圖:鄭心芸攝)相關個股元大期6023近年來投資市場波動越來越明顯 ### 省略內文 ### 都有非常巨大的助益。,[]
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北市政府今日下午也跨局處召開「外送人員保障專案會議」。圖／台北市勞動局提供,[]
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文｜李育材攝影｜李育材全文朗讀00:00 / 00:00君鴻飯店快速遭法拍 ### 省略內文 ### 亦即本次拍賣事件均屬合法。更新時間｜2019.10.10 09:14,[]
3,4,https://www.chinatimes.com/realtimenews/20190917000619-260106,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。 (中國時報),[]
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9HN03.jsp?crtid=PTD&filenm=ECE5DAECAF563DA171E34B03FC5185742FC90F879191D5DF,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家親108年度司家催字第58號附件：主旨：公告本院108年度司家催字第58號公示催告事件裁定。依據：家事事件法第137條、第138條、第139條準用第130條第3項至第5項、臺灣地區及大陸地區人民關係條例第68條第1項、第3項、退除役官兵死亡無人繼承遺產管理辦法第6條。公告事項：一、本案遺產管理人為國軍退除役官兵輔導委員會屏東榮譽國民之家（住址：屏東縣內埔鄉○○路000號）。二、本件係對被繼承人陳世恒（男、民國16年10月6日生、身分證統一編號：F10****885號、108年8月29日死亡、生前最後住所地：屏東縣內埔鄉○○路000號）之繼承人、債權人及受遺贈人為公示催告。三、被繼承人陳世恒之繼承人（除大陸地區人民應依臺灣地區與大陸地區人民關係條例之規定外）、債權人及受遺贈人 ### 省略內文 ### 僅得就賸餘遺產行使其權利。五、本件裁定正本黏貼於本院公告處。核章欄：書記官科長司法事務官書記官：沈君融股別：親,[]


In [3]:
c_list = df["content"].tolist()
content = []

for c in c_list:
    c = c.split("### 省略內文 ###")
    tmp = []
    prog = re.compile(r"\s+")
    for s in c:
        tokens = prog.split(
            unicodedata.normalize("NFKC", s.strip()))
        tmp.append("".join(tokens))
    content.append(tmp)

## Get crawled files

In [4]:
DIR_PATH = os.path.abspath(
    f"./data/news"
)
FILES = sorted(os.listdir(DIR_PATH))
FILES.sort(key=len, reverse=False)

FILES = [f"{DIR_PATH}/{path}" for path in FILES]

corpus = []

for p in FILES:
    with open(p, "r") as f:
        corpus.append(f.readline())

In [5]:
f"Max Document Length: {max([len(c) for c in corpus])}"

'Max Document Length: 6974'

In [6]:
u_name = df["name"].tolist()
merged = zip(corpus, u_name)

## Merge crawled with labels

In [7]:
final_df = pd.DataFrame(
    {
        'corpus' : corpus,
        'labels' : u_name
    }
)

In [8]:
final_df.head()

,corpus,labels
0,"近年來投資市場波動越來越明顯,追求低波動、絕對報酬的量化交易備受注目。專家表示,採用量化交易策略投資台股,不管是處於多頭或是空頭市場,績效及波動度均可領跑大盤,甚至比國內投資台股的股票型基金及 ETF 的波動率還低,表現也更為穩定。\n",[]
1,"10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件,北市府勞動局認定業者未依職業安全衛生法第37條第2項第1款規定於8小時內向勞動檢查機構通報,開罰Uber Eats 30萬元。\n",[]
2,"君鴻酒店業者控訴高雄地院強制法拍龐大資產,是為了讓財團得利。君鴻業主表示,拍得君鴻的凱德唐公司要求85大樓管委會,在今年8月21日晚上11點後,將酒店斷水斷電,還讓電梯、緊急升降梯停擺,甚至將緊急逃生門鎖死,無視君鴻酒店於點交前仍可合法使用,還有損仍在大樓內辦公,且不點交的福勵、樂活、富勝3家公司員工權益,導致員工無法上班,客戶無法進入,以致公司經營困難、損失巨大。這3家公司緊急反應給高雄地院執行處,沒想到高雄地院竟回覆這些事項非該處所得置問,叫可憐員工情何以堪?\n",[]
3,"404錯誤 - 中時電子報 中時電子報 關閉 新冠肺炎 即時 展開子選單 即時首頁 政治 生活 社會 旅遊 娛樂 體育 財經 國際 兩岸 科技 軍事 人物 政治 展開子選單 政治首頁 總覽 言論 展開子選單 首頁 總覽 無色覺醒 中時社論 旺報社評 工商社論 短評 名家 星期論壇 觀點 徵文 史話 百川 生活 展開子選單 生活首頁 總覽 Campus 娛樂 展開子選單 娛樂首頁 總覽 新聞 日韓 孝親報報 財經 展開子選單 財經首頁 即時 要聞 證券 櫃買 觀點 全球 金融 科技 產業 特刊 解盤 理財 自貿區 報稅 社會 展開子選單 社會首頁 總覽 刑案 意外 男女 暖聞 萬象 一瞬間 法律 地方 寶島 展開子選單 寶島首頁 總覽 台北基宜花 新北金馬 桃園竹苗 台中彰投 台南雲嘉 高雄屏東澎 話題 展開子選單 話題首頁 總覽 日常 時事 正妹 星奇 搜奇 歷史 好書 雜誌 FB 挖新鮮 有影 展開子選單 有影首頁 總覽 搜奇 生活 娛樂 時事 社會 中天亞洲台 追劇 國際 展開子選單 國際首頁 總覽 時事 趨勢 科技 新知 健康 名流 奇聞 軍事 展開子選單 軍事首頁 總覽 空軍 海軍 陸軍 太空 網路 戰史 戰略 兩岸 展開子選單 兩岸首頁 總覽 時政 台海 廣州 錢潮 環保 話題 深度 名人 時尚 展開子選單 時尚首頁 總覽 時尚 美妝 星光 風格 運勢 消費 啵滋真心話 體育 展開子選單 體育首頁 總覽 時來運轉 亞運 排球 高爾夫 MLB 棒球 NBA 籃球 足球 網球 其他 科技 展開子選單 科技首頁 總覽 智慧手機 私藏密技 電腦應用 網路社群 新知趣聞 消費情報 娛樂遊戲 產業新聞 數位服務 科學前沿 玩食 展開子選單 玩食首頁 總覽 專輯 展開子選單 首頁 總覽 時事 名人 全球 運動 災難 八卦 星聞 局勢 節日 新知 生活 軍事 首頁 404錯誤 抱歉!您所查詢的資料,目前無法找到任何頁面! 可能是因為: 您輸入了錯誤的網址 此頁面已被移除或不存在",[]
4,"家事事件公告 查尋條件: 列印 例稿名稱:臺灣屏東地方法院公示催告公告 發文日期:中華民國108年9月20日 發文字號:屏院進家親108年度司家催字第58號 附件: 主旨:公告本院108年度司家催字第58號公示催告事件裁定。 依據:家事事件法第137 條、第138 條、第139 條準用第130 條 第3 項至第5 項、臺灣地區及大陸地區人民關係條例第68 條第1 項、第3 項、退除役官兵死亡無人繼承遺產管理辦 法第6 條。 公告事項: 一、本案遺產管理人為國軍退除役官兵輔導委員會屏東榮譽國 民之家(住址:屏東縣內埔鄉○○路000 號)。 二、本件係對被繼承人陳世恒(男、民國16年10月6 日生、身 分證統一編號:F10****885號、108 年8 月29日死亡、生 前最後住所地:屏東縣內埔鄉○○路000 號)之繼承人、 債權人及受遺贈人為公示催告。 三、被繼承人陳世恒之繼承人(除大陸地區人民應依臺灣地區 與大陸地區人民關係條例之規定外)、債權人及受遺贈人 ,應自本公示催告揭示於本院公告處、司法院資訊網路之 日起,1 年2 個月內,向遺產管理人承認繼承、報明其債 權及為是否願受遺贈之聲明。 四、繼承人不於上述期間內承認繼承,於清償債權並交付遺贈 物後,如有剩餘,歸屬國庫。債權人、受遺贈人未於上述 期間內報明、聲明者,僅得就賸餘遺產行使其權利。 五、本件裁定正本黏貼於本院公告處。 核章欄: 書記官 科長 司法事務官 書記官:沈君融 股別:親 司法院 資訊管理處 製作。 對於本系統功能有任何建議, 歡迎來信 。 如欲查詢上開公告之裁定,請逕至本院「裁判書查詢」網頁查詢。 對公告內容有疑義者,請逕向承辦法院書記官洽詢,或電洽司法院。",[]


## Get all names

In [9]:
name_list = df["name"].tolist()
name_list = [ast.literal_eval(name) for name in name_list]
print(f"Max Number of Names: {max([len(ns) for ns in name_list])}")

x = {idx: ns for idx, ns in enumerate(name_list) if len(ns) == 0}
print(f"Number of documents without label: {len(x)}")

Max Number of Names: 15
Number of documents without label: 4651


In [10]:
unique_names = list({n for name in name_list for n in name})
print(f"The number of unique names: {len(unique_names)}")
print(unique_names)

The number of unique names: 830
['郭盈志', '周榮華', '張道銘', '詹信夫', '王柏森', '周麗真', '黃文哲', '簡以珍', '蔡豐益', '林清井', '詹雅琳', '夏子茵', '洪正義', '邱希庚', '張治忠', '沈大祥', '姜維池', '葉嘉雄', '林育佑', 'Denis Katana', '簡志霖', '黃啟禎', '陳學敏', '陳威愷', '黃哲修', '郭明枝', '祁明昕', '陳効亮', '黃俊策', '徐少東', '葉文豪', '林坤星', '黃奕霖', '崔明禮', '羅雅美', '施允澤', '施金樹', '馬乃林', '黃建強', '黃建彰', '謝春發', '林奕如', '陸佩啟', '詹博翔', '何煖軒', '林文章', '呂紹甫', '陳致中', '王韻茹', '陳建閣', '林睿耆', '徐世立', '黃柏維', '鄭少熙', '楊善淵', '吳松麟', '黃淑頻', '游世一', '陳寶生', '陳文政', '蔡景德', '周雲楠', '蔡思庭', '黃博鴻', '許士耘', '蘇清俊', '林義傑', '王洪烈', '林國欽', '倪菲爾', '黃文焱', '彭文潞', '張嘉玲', '楊永昌', '陳水扁', '曾杭皆', '王羿雄', '李哲銘', '吳秋華', '林偉琳', '洪博文', '趙瑩潔', '王威仁', '王姝茵', '魏君婷', '陳世坤', '鄭又仁', '陳震歐', '鄧福鈞', '陳榮祥', '謝淑美', '高嘉斌', '張簡復中', '蔡世誠', '黃國清', '江東原', '蘇怡寧', '李宛霞', '李明華', '紀雅玲', '黃春發', '何宗英', '孫岳澤', '徐豪雄', '趙宏翰', '王隆昌', '葉麗貞', '雷俊玲', '李瑞廷', '張語庭', '陳傳志', '陳孝仁', '曾國財', '高盟傑', '王春甡', '林健標', '徐金龍', '楊富巖', '繆竹怡', '林振宏', '林耿宏', '黃子愛', '林嘉凌', '曾煥彰', '陳武騰', '陳文宏', 'Alan Lim', '陳彥廷', '楊文海', '黃薪哲', '王鴻財', '陳柏維', '陳西元', '李政勳', '吳宇倫', '朱國榮', 

## NER Tagger

In [ ]:
ws = WS("./ckip")
ner = NER("./ckip")
pos = POS("./ckip")
word_sentence_list = ws(corpus)
pos_sentence_list = pos(corpus)

# entity_sentence_list = ner(word_sentence_list, pos_sentence_list)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [ ]:
print(len(word_sentence_list))
print(len(pos_sentence_list))
